In [1]:
import cv2
from PIL import Image, ImageOps
import argparse
import os
import glob
import matplotlib.pyplot as plt
import multiprocessing as mp
from joblib import Parallel, delayed
import pydicom
import time
import numpy as np
import torch
import random
import pandas as pd
from pydicom.pixel_data_handlers import apply_windowing
import imageio
import time
import shutil

In [ ]:
img = cv2.imread

In [ ]:
import pydicom

dcm = pydicom.dcmread('path/to/dicom/file.dcm')
width = dcm.Rows
height = dcm.Columns
img = dcm.pixel_array


## 3 CLASS

In [3]:
mias = pd.read_csv("E:/WORKBASE/Project-rsna-breast-cancer-detection/DATA_STANDARD/CSV 3 CLASS/Mias.csv", index_col='Unnamed: 0')

In [4]:
ddsm = pd.read_csv("E:\WORKBASE\Project-rsna-breast-cancer-detection\DATA_STANDARD\CSV 3 CLASS\meta_ddsm.csv", index_col='Unnamed: 0')

In [5]:
cdd = pd.read_csv("E:\WORKBASE\Project-rsna-breast-cancer-detection\DATA_STANDARD\CSV 3 CLASS\meta_cdd.csv", index_col='Unnamed: 0')

In [38]:
cmmd = pd.read_csv("E:/WORKBASE/Project-rsna-breast-cancer-detection/TheChineseMammographyDatabase/TheChineseMammographyDatabase/meta_main.csv", index_col='Unnamed: 0')
cmmd.path[0]

'D:/OneDrive - Industrial University of HoChiMinh City/WORKBASE/Project-rsna-breast-cancer-detection/TheChineseMammographyDatabase/TheChineseMammographyDatabase/CMMD/D1-0001/07-18-2010-NA-NA-79377/1.000000-NA-70244\\1-1.dcm'

In [7]:
cmmd.path = cmmd.path.apply(lambda x: x.replace('D:/OneDrive - Industrial University of HoChiMinh City', 'E:'))

In [40]:
cmmd.cancer.value_counts()

cancer
1    2633
0    1108
Name: count, dtype: int64

In [8]:
cmmd.image_id = cmmd.image_id.apply(lambda x: x.replace('_', '-'))

In [9]:
cmmd

,index,patient_id,image_id,laterality,cancer,path,view
0,0,D1-0001,D1-0001-1-1,R,0,E:/WORKBASE/Project-rsna-breast-cancer-detecti...,MLO
1,1,D1-0001,D1-0001-1-2,R,0,E:/WORKBASE/Project-rsna-breast-cancer-detecti...,CC
2,2,D1-0002,D1-0002-1-1,L,0,E:/WORKBASE/Project-rsna-breast-cancer-detecti...,MLO
3,3,D1-0002,D1-0002-1-2,L,0,E:/WORKBASE/Project-rsna-breast-cancer-detecti...,CC
4,4,D1-0003,D1-0003-1-1,L,0,E:/WORKBASE/Project-rsna-breast-cancer-detecti...,MLO
...,...,...,...,...,...,...,...
3736,3736,D2-0747,D2-0747-1-4,R,1,E:/WORKBASE/Project-rsna-breast-cancer-detecti...,CC
3737,3737,D2-0748,D2-0748-1-3,R,1,E:/WORKBASE/Project-rsna-breast-cancer-detecti...,MLO
3738,3738,D2-0748,D2-0748-1-4,R,1,E:/WORKBASE/Project-rsna-breast-cancer-detecti...,CC
3739,3739,D2-0749,D2-0749-1-1,L,1,E:/WORKBASE/Project-rsna-breast-cancer-detecti...,MLO


In [10]:
rsna = pd.read_csv("E:\WORKBASE\Project-rsna-breast-cancer-detection\DATA_STANDARD\RSNA.csv", index_col='Unnamed: 0')

> MOVE IMAGE MISTASK

In [11]:
from remove_image_views import *


In [25]:
mv_cmmd = move_image(cmmd, path='E:/WORKBASE/Project-rsna-breast-cancer-detection/DATA_STANDARD/Croped_CMMD')
mv_cmmd.my_method(cmmd.index)
# mv_cmmd.process_move(0)

In [ ]:
mv_rsna = move_image(rsna, path='E:/WORKBASE/Project-rsna-breast-cancer-detection/DATA_STANDARD/Croped_RSNA')
mv_rsna.my_method(rsna.index)
# mv_cmmd.process_move(0)

In [104]:
def procees_dcm(path):
    
    dicom = pydicom.dcmread(path)
    # try : 
    img = dicom.pixel_array
    # img = apply_windowing(img, dicom)
    img = (img - img.min()) / (img.max() - img.min())

    if dicom.PhotometricInterpretation == "MONOCHROME1":  
        img = 1 - img
    image = np.uint8(img * 255)

    return image

In [108]:
for id in cmmd.index:
    par = cmmd.patient_id[id]
    lat = cmmd.laterality[id]
    view = cmmd.view[id]
    file_name = f"E:/WORKBASE/Project-rsna-breast-cancer-detection/DATA_STANDARD/Croped_CMMD/{cmmd.view[id]}_{cmmd.image_id[id]}.png"
    if os.path.exists(file_name) ==False:
        imageio.imwrite(f'E:/WORKBASE/Project-rsna-breast-cancer-detection/DATA_STANDARD/Croped_CMMD/{view}_{cmmd.image_id[id]}.png', procees_dcm(cmmd.path[id]))

> ADD PATH

In [12]:
mias['Path'] = mias.image_id.apply(lambda x: f'Data_image/Mias/{x}.png')

In [13]:
ddsm['Path'] = ddsm.apply(lambda row: f'Data_image/{row["patient_id"]}/{row["view"]}/{row["laterality"]}/{row["view"]}_{row["laterality"]}_{row["image_id"]}.png', axis=1)

In [14]:
cdd['Path'] = cdd.apply(lambda row: f'Data_image/{row["patient_id"]}/{row["view"]}/{row["laterality"]}/{row["view"]}_{row["laterality"]}_{row["image_id"]}.png', axis=1)

In [15]:
rsna['Path'] = rsna.apply(lambda row: f'Data_image/{row["patient_id"]}/{row["view"]}/{row["laterality"]}/{row["view"]}_{row["laterality"]}_{row["image_id"]}.png', axis=1)

In [16]:
cmmd['Path'] = cmmd.apply(lambda row: f'Data_image/{row["patient_id"]}/{row["view"]}/{row["laterality"]}/{row["view"]}_{row["laterality"]}_{row["image_id"]}.png', axis=1)

> ADD TYPE DATA

In [17]:
mias['Type'] = mias.image_id.apply(lambda x: 'Mias')
ddsm['Type'] = ddsm.image_id.apply(lambda x: 'ddsm')
cdd['Type'] = cdd.image_id.apply(lambda x: 'cdd')
rsna['Type'] = rsna.image_id.apply(lambda x: 'rsna')
cmmd['Type'] = cmmd.image_id.apply(lambda x: 'cmmd')

In [18]:
def fix(data):
    dc = {0: 0, 1: 0, 2: 1}
    if 'status' in data.columns:
        data.status = data.status.apply(lambda x: dc[x])
    else:
        data = data.rename(columns = {'cancer':'status'})
    return data
mias = fix(mias)
cmmd = fix(cmmd)
ddsm = fix(ddsm)
rsna = fix(rsna)
cdd = fix(cdd)

> Format data

In [19]:
mias = mias[['image_id', 'Path', 'Type', 'status']]
cdd = cdd[['image_id', 'Path', 'Type', 'status']]
ddsm = ddsm[['image_id', 'Path', 'Type', 'status']]
rsna = rsna[['image_id', 'Path', 'Type', 'status']]
cmmd = cmmd[['image_id', 'Path', 'Type', 'status']]

> Split data

In [20]:
print(f'Size Mias {len(mias)}')
print(f'Size cdd {len(cdd)}')
print(f'Size ddsm {len(ddsm)}')
print(f'Size rsna {len(rsna)}')
print(f'Size cmmd {len(cmmd)}')

Size Mias 322
Size cdd 1066
Size ddsm 7808
Size rsna 49968
Size cmmd 3741


In [21]:
print(f'Detail mias {mias.status.value_counts()}')
print(f'Detail cdd {cdd.status.value_counts()}')
print(f'Detail ddsm {ddsm.status.value_counts()}')
print(f'Detail rsna {rsna.status.value_counts()}')
print(f'Detail cmmd {cmmd.status.value_counts()}')

Detail mias status
0    270
1     52
Name: count, dtype: int64
Detail cdd status
0    702
1    364
Name: count, dtype: int64
Detail ddsm status
0    5092
1    2716
Name: count, dtype: int64
Detail rsna status
0    48932
1     1036
Name: count, dtype: int64
Detail cmmd status
1    2633
0    1108
Name: count, dtype: int64


In [26]:
1036*0.1

103.60000000000001

In [27]:
1108*0.1

110.80000000000001

TEST

In [22]:
def get10(data, ts1, ts2):
    d0 = data[data.status==0].sample(ts1)
    d1 = data[data.status==1].sample(ts2)
    # d2 = data[data.status==2].sample(ts3)
    return pd.concat([d0, d1]).sample(frac=1).reset_index(drop=True)
def concatdata(list_data):
    list_idimage = []
    for data, name,  ts1, ts2 in list_data:
        # if len(meta) == 0:
        meta = get10(data, ts1, ts2)
        list_idimage = list_idimage + list(meta.image_id)
        meta.to_csv(f'E:/WORKBASE/Project-rsna-breast-cancer-detection/META_DATA/2CLASS/Test_{name}.csv')
    return list_idimage 

In [23]:
list_idimage = concatdata(
    [(mias, 'mias', 20, 10), 
     (ddsm, 'ddsm', 271*2, 271), 
     (cdd, 'cdd', 2*36, 36), 
     (rsna, 'rsna', 2*103, 103), 
     (cmmd, 'cmmd', 2*110, 110)])

In [27]:
from sklearn.model_selection import KFold
import random

In [32]:
def process_split_rsna(meta, name_data = "RONG"):
    meta = meta.reset_index()
    meta = meta[~meta.image_id.isin(list_idimage)]
    meta = meta[['image_id'	,'Path'	,'Type',	'status']]
    # Lặp lại huấn luyện và đánh giá trên các phần được chia
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    nocancer_data = meta[meta.status == 0].reset_index()
    cancer_data = meta[meta.status ==1].reset_index()

    # Lặp lại huấn luyện và đánh giá trên các phần được chia
    remain = list(nocancer_data.index)
    sums = 0
    for id, (train_index, val_index) in enumerate(kf.split(cancer_data)):
        # Tạo dữ liệu huấn luyện và dữ liệu đánh giá
        os.makedirs(f'E:/WORKBASE/Project-rsna-breast-cancer-detection/META_DATA/2CLASS/{name_data}/{id}-fold')
        train_index = list(train_index)
        val_index = list(val_index)
        plus = 0
        if name_data == 'RSNA':
            plus = 1000
        id_list_train = random.sample(remain, k=len(train_index) + plus)

        train_data = cancer_data.iloc[train_index]
        train_data = pd.concat([train_data, nocancer_data.iloc[id_list_train]]).sample(frac=1).reset_index(drop=True)
        remain = set(remain) - set(id_list_train)
        # print(train_data.status.value_counts())
        
        id_list_val = random.sample(remain, k=len(val_index))

        val_data = cancer_data.iloc[val_index]
        val_data = pd.concat([val_data, nocancer_data.iloc[id_list_val]]).sample(frac=1).reset_index(drop=True)
        remain = set(remain) - set(id_list_val)
        # print(val_data.status.value_counts())
        train_data.to_csv(f'E:/WORKBASE/Project-rsna-breast-cancer-detection/META_DATA/2CLASS/{name_data}/{id}-fold/train.csv')
        val_data.to_csv(f'E:/WORKBASE/Project-rsna-breast-cancer-detection/META_DATA/2CLASS/{name_data}/{id}-fold/val.csv')
        ## description
        with open(f'E:/WORKBASE/Project-rsna-breast-cancer-detection/META_DATA/2CLASS/{name_data}/{id}-fold/description.txt', 'a') as f:
            f.writelines(f'Size of train: {len(train_data)} \n\n Size of val: {len(val_data)} \n\n Number of per dataset: \n {train_data.Type.value_counts()}\n\n Status of set train: \n{train_data.status.value_counts()}\n\n Status of set val: \n{val_data.status.value_counts()}')
        print(len(train_data), len(val_data))
        print(train_data.Type.value_counts())
        print(train_data.status.value_counts())
        print('++++++++++++++++++')

In [34]:
def process_split(meta, name_data = "RONG"):
    meta = meta.reset_index()
    meta = meta[~meta.image_id.isin(list_idimage)]
    meta = meta[['image_id'	,'Path'	,'Type',	'status']]
    # Lặp lại huấn luyện và đánh giá trên các phần được chia
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    for id, (train_index, val_index) in enumerate(kf.split(meta)):
        # Tạo dữ liệu huấn luyện và dữ liệu đánh giá
        os.makedirs(f'E:/WORKBASE/Project-rsna-breast-cancer-detection/META_DATA/2CLASS/{name_data}/{id}-fold')
        
        train_data = meta.iloc[train_index]
        val_data = meta.iloc[val_index]
        train_data.to_csv(f'E:/WORKBASE/Project-rsna-breast-cancer-detection/META_DATA/2CLASS/{name_data}/{id}-fold/train.csv')
        val_data.to_csv(f'E:/WORKBASE/Project-rsna-breast-cancer-detection/META_DATA/2CLASS/{name_data}/{id}-fold/val.csv')
        ## description
        with open(f'E:/WORKBASE/Project-rsna-breast-cancer-detection/META_DATA/2CLASS/{name_data}/{id}-fold/description.txt', 'a') as f:
            f.writelines(f'Size of train: {len(train_data)} \n\n Size of val: {len(val_data)} \n\n Number of per dataset: \n {train_data.Type.value_counts()}\n\n Status of set train: \n{train_data.status.value_counts()}')
        print(len(train_data), len(val_data))
        print(train_data.Type.value_counts())
        print(train_data.status.value_counts())
        print('++++++++++++++++++')

In [37]:
cmmd.status.value_counts()

status
1    2633
0    1108
Name: count, dtype: int64

In [42]:
process_split(cdd,'CDD')

759 190
Type
cdd    759
Name: count, dtype: int64
status
0    502
1    257
Name: count, dtype: int64
++++++++++++++++++
759 190
Type
cdd    759
Name: count, dtype: int64
status
0    500
1    259
Name: count, dtype: int64
++++++++++++++++++
759 190
Type
cdd    759
Name: count, dtype: int64
status
0    494
1    265
Name: count, dtype: int64
++++++++++++++++++
759 190
Type
cdd    759
Name: count, dtype: int64
status
0    505
1    254
Name: count, dtype: int64
++++++++++++++++++
760 189
Type
cdd    760
Name: count, dtype: int64
status
0    499
1    261
Name: count, dtype: int64
++++++++++++++++++


In [41]:
process_split(mias,'MIAS')

233 59
Type
Mias    233
Name: count, dtype: int64
status
0    200
1     33
Name: count, dtype: int64
++++++++++++++++++
233 59
Type
Mias    233
Name: count, dtype: int64
status
0    201
1     32
Name: count, dtype: int64
++++++++++++++++++
234 58
Type
Mias    234
Name: count, dtype: int64
status
0    198
1     36
Name: count, dtype: int64
++++++++++++++++++
234 58
Type
Mias    234
Name: count, dtype: int64
status
0    203
1     31
Name: count, dtype: int64
++++++++++++++++++
234 58
Type
Mias    234
Name: count, dtype: int64
status
0    198
1     36
Name: count, dtype: int64
++++++++++++++++++


In [36]:
process_split(cmmd,'CMMD')

2728 683
Type
cmmd    2728
Name: count, dtype: int64
status
1    2013
0     715
Name: count, dtype: int64
++++++++++++++++++
2729 682
Type
cmmd    2729
Name: count, dtype: int64
status
1    2012
0     717
Name: count, dtype: int64
++++++++++++++++++
2729 682
Type
cmmd    2729
Name: count, dtype: int64
status
1    2014
0     715
Name: count, dtype: int64
++++++++++++++++++
2729 682
Type
cmmd    2729
Name: count, dtype: int64
status
1    2033
0     696
Name: count, dtype: int64
++++++++++++++++++
2729 682
Type
cmmd    2729
Name: count, dtype: int64
status
1    2020
0     709
Name: count, dtype: int64
++++++++++++++++++


In [35]:
process_split(ddsm,'DDSM')

5596 1399
Type
ddsm    5596
Name: count, dtype: int64
status
0    3619
1    1977
Name: count, dtype: int64
++++++++++++++++++
5596 1399
Type
ddsm    5596
Name: count, dtype: int64
status
0    3636
1    1960
Name: count, dtype: int64
++++++++++++++++++
5596 1399
Type
ddsm    5596
Name: count, dtype: int64
status
0    3649
1    1947
Name: count, dtype: int64
++++++++++++++++++
5596 1399
Type
ddsm    5596
Name: count, dtype: int64
status
0    3638
1    1958
Name: count, dtype: int64
++++++++++++++++++
5596 1399
Type
ddsm    5596
Name: count, dtype: int64
status
0    3658
1    1938
Name: count, dtype: int64
++++++++++++++++++


In [29]:
process_split_rsna(rsna,'RSNA')

2492 374
Type
rsna    2492
Name: count, dtype: int64
status
0    1746
1     746
Name: count, dtype: int64
++++++++++++++++++
2492 374
Type
rsna    2492
Name: count, dtype: int64
status
0    1746
1     746
Name: count, dtype: int64
++++++++++++++++++
2492 374
Type
rsna    2492
Name: count, dtype: int64
status
0    1746
1     746
Name: count, dtype: int64
++++++++++++++++++
2494 372
Type
rsna    2494
Name: count, dtype: int64
status
0    1747
1     747
Name: count, dtype: int64
++++++++++++++++++
2494 372
Type
rsna    2494
Name: count, dtype: int64
status
0    1747
1     747
Name: count, dtype: int64
++++++++++++++++++


C:\Users\nguye\AppData\Local\Temp\ipykernel_20556\1156713600.py:28: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  id_list_val = random.sample(remain, k=len(val_index))
C:\Users\nguye\AppData\Local\Temp\ipykernel_20556\1156713600.py:21: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  id_list_train = random.sample(remain, k=len(train_index) + plus)
C:\Users\nguye\AppData\Local\Temp\ipykernel_20556\1156713600.py:28: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  id_list_val = random.sample(remain, k=len(val_index))
C:\Users\nguye\AppData\Local\Temp\ipykernel_20556\1156713600.py:21: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  id_list_train = random.sample(remain, k=len(train_index) + plus)
C:\Users\nguye\AppData\Local\T

In [32]:
meta = pd.concat([mias, ddsm, cdd, rsna, cmmd]).sample(frac=1).reset_index(drop=True)
meta = meta[~meta.image_id.isin(list_idimage)]
meta

,image_id,Path,Type,status
0,1505346563,Data_image/18790/CC/L/CC_L_1505346563.png,rsna,0
1,4174DDSM,Data_image/1728DDSM/CC/R/CC_R_4174DDSM.png,ddsm,1
2,1988072350,Data_image/47985/MLO/R/MLO_R_1988072350.png,rsna,0
3,1523041670,Data_image/36847/MLO/L/MLO_L_1523041670.png,rsna,0
4,629070079,Data_image/42886/MLO/L/MLO_L_629070079.png,rsna,0
...,...,...,...,...
62900,D1-0664-1-1,Data_image/D1-0664/MLO/R/MLO_R_D1-0664-1-1.png,cmmd,1
62901,2919DDSM,Data_image/0068DDSM/MLO/R/MLO_R_2919DDSM.png,ddsm,1
62902,1545282521,Data_image/20066/MLO/L/MLO_L_1545282521.png,rsna,0
62903,2087599778,Data_image/28916/MLO/R/MLO_R_2087599778.png,rsna,0


In [84]:
from sklearn.model_selection import KFold
import random

In [132]:
meta = meta.reset_index()

In [133]:
meta = meta[['image_id'	,'Path'	,'Type',	'status']]

In [134]:
meta.status.value_counts()

status
0    55038
1     6263
Name: count, dtype: int64

In [144]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
nocancer_data = meta[meta.status == 0].reset_index()
cancer_data = meta[meta.status ==1].reset_index()

# Lặp lại huấn luyện và đánh giá trên các phần được chia
remain = list(nocancer_data.index)
sums = 0
for id, (train_index, val_index) in enumerate(kf.split(cancer_data)):
    # Tạo dữ liệu huấn luyện và dữ liệu đánh giá
    os.makedirs(f'E:/WORKBASE/Project-rsna-breast-cancer-detection/META_DATA/2CLASS/{id}-fold')
    train_index = list(train_index)
    val_index = list(val_index)
    
    id_list_train = random.sample(remain, k=len(train_index) + 1000)

    train_data = cancer_data.iloc[train_index]
    train_data = pd.concat([train_data, nocancer_data.iloc[id_list_train]]).sample(frac=1).reset_index(drop=True)
    remain = set(remain) - set(id_list_train)
    # print(train_data.status.value_counts())
    
    id_list_val = random.sample(remain, k=len(val_index) + 100)

    val_data = cancer_data.iloc[val_index]
    val_data = pd.concat([val_data, nocancer_data.iloc[id_list_val]]).sample(frac=1).reset_index(drop=True)
    remain = set(remain) - set(id_list_val)
    
 
    # print(val_data.status.value_counts())
    train_data.to_csv(f'E:/WORKBASE/Project-rsna-breast-cancer-detection/META_DATA/2CLASS/{id}-fold/train.csv')
    val_data.to_csv(f'E:/WORKBASE/Project-rsna-breast-cancer-detection/META_DATA/2CLASS/{id}-fold/val.csv')
    ## description
    with open(f'E:/WORKBASE/Project-rsna-breast-cancer-detection/META_DATA/2CLASS/{id}-fold/description.txt', 'a') as f:
        f.writelines(f'Size of train: {len(train_data)} \n\n Size of val: {len(val_data)} \n\n Number of per dataset: \n {train_data.Type.value_counts()}\n\n Status of set train: \n{train_data.status.value_counts()}\n\n Status of set val: \n{val_data.status.value_counts()}')
    print(len(train_data), len(val_data))
    print(train_data.Type.value_counts())
    print(train_data.status.value_counts())
    print('++++++++++++++++++')

C:\Users\nguye\AppData\Local\Temp\ipykernel_2576\2194692045.py:21: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  id_list_val = random.sample(remain, k=len(val_index) + 100)
C:\Users\nguye\AppData\Local\Temp\ipykernel_2576\2194692045.py:14: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  id_list_train = random.sample(remain, k=len(train_index) + 1000)
C:\Users\nguye\AppData\Local\Temp\ipykernel_2576\2194692045.py:21: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  id_list_val = random.sample(remain, k=len(val_index) + 100)
C:\Users\nguye\AppData\Local\Temp\ipykernel_2576\2194692045.py:14: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  id_list_train = random.sample(remain, k=len(train_index) + 1000)
C:\Users\nguye\AppData

11020 2606
Type
rsna    6073
ddsm    2452
cmmd    2112
cdd      323
Mias      60
Name: count, dtype: int64
status
0    6010
1    5010
Name: count, dtype: int64
++++++++++++++++++
11020 2606
Type
rsna    6057
ddsm    2485
cmmd    2083
cdd      338
Mias      57
Name: count, dtype: int64
status
0    6010
1    5010
Name: count, dtype: int64
++++++++++++++++++
11020 2606
Type
rsna    6064
ddsm    2442
cmmd    2133
cdd      312
Mias      69
Name: count, dtype: int64
status
0    6010
1    5010
Name: count, dtype: int64
++++++++++++++++++
11022 2604
Type
rsna    6044
ddsm    2495
cmmd    2107
cdd      312
Mias      64
Name: count, dtype: int64
status
0    6011
1    5011
Name: count, dtype: int64
++++++++++++++++++
11022 2604
Type
rsna    6076
ddsm    2408
cmmd    2148
cdd      326
Mias      64
Name: count, dtype: int64
status
0    6011
1    5011
Name: count, dtype: int64
++++++++++++++++++


C:\Users\nguye\AppData\Local\Temp\ipykernel_2576\2194692045.py:14: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  id_list_train = random.sample(remain, k=len(train_index) + 1000)
C:\Users\nguye\AppData\Local\Temp\ipykernel_2576\2194692045.py:21: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  id_list_val = random.sample(remain, k=len(val_index) + 100)


In [107]:
for i in id_list_train:
    print(i)
    s = nocancer_data.iloc[i]

38453
23973
55774


IndexError: single positional indexer is out-of-bounds

In [110]:
nocancer_data.iloc[55774]

IndexError: single positional indexer is out-of-bounds

In [112]:
nocancer_data = nocancer_data[['image_id'	,'Path'	,'Type'	,'status']]

In [115]:
nocancer_data.iloc[55774]

IndexError: single positional indexer is out-of-bounds

In [116]:
nocancer_data.index

Index([    0,     2,     3,     4,     5,     6,     7,     8,     9,    10,
       ...
       61289, 61290, 61291, 61292, 61293, 61294, 61295, 61298, 61299, 61300],
      dtype='int64', length=55038)

In [79]:
df = pd.read_csv(r"E:\WORKBASE\Project-rsna-breast-cancer-detection\META_DATA\2CLASS\0-fold\train.csv")
df[df.status == 1]

,Unnamed: 0,index,image_id,Path,Type,status
1,1,1,4174DDSM,Data_image/1728DDSM/CC/R/CC_R_4174DDSM.png,ddsm,1
18,24,26,D1-1135-1-2,Data_image/D1-1135/CC/R/CC_R_D1-1135-1-2.png,cmmd,1
20,26,28,3497DDSM,Data_image/0219DDSM/MLO/L/MLO_L_3497DDSM.png,ddsm,1
23,30,32,D2-0709-1-2,Data_image/D2-0709/CC/L/CC_L_D2-0709-1-2.png,cmmd,1
32,43,45,D1-1721-1-1,Data_image/D1-1721/MLO/L/MLO_L_D1-1721-1-1.png,cmmd,1
...,...,...,...,...,...,...
4983,6227,6407,mdb249,Data_image/Mias/mdb249.png,Mias,1
4986,6231,6411,P266-R-DM-CC,Data_image/266CDD/CC/R/CC_R_P266-R-DM-CC.png,cdd,1
4999,6250,6431,D1-0428-1-1,Data_image/D1-0428/MLO/L/MLO_L_D1-0428-1-1.png,cmmd,1
5002,6253,6434,1982155258,Data_image/26664/MLO/R/MLO_R_1982155258.png,rsna,1


In [93]:
meta.iloc[cancer_data.index].status.value_counts()

status
1    6263
Name: count, dtype: int64

In [39]:
len(balanced_nocancer_data)

11005

## 2 CLASS

We have 5 dataset: DDSM, RSNA, CMMD, CDD, Mias
+ So we will collect status 0, 1 --> 0 and 2 --> 1

In [61]:
mias

,image_id,Path,Type,status
0,mdb001,Data_image/Mias/mdb001.png,Mias,1
1,mdb002,Data_image/Mias/mdb002.png,Mias,1
2,mdb003,Data_image/Mias/mdb003.png,Mias,0
3,mdb004,Data_image/Mias/mdb004.png,Mias,0
4,mdb005,Data_image/Mias/mdb005.png,Mias,1
...,...,...,...,...
317,mdb318,Data_image/Mias/mdb318.png,Mias,0
318,mdb319,Data_image/Mias/mdb319.png,Mias,0
319,mdb320,Data_image/Mias/mdb320.png,Mias,0
320,mdb321,Data_image/Mias/mdb321.png,Mias,0


In [62]:
ddsm

,image_id,Path,Type,status
0,0DDSM,Data_image/0029DDSM/CC/L/CC_L_0DDSM.png,ddsm,1
1,1DDSM,Data_image/0029DDSM/MLO/L/MLO_L_1DDSM.png,ddsm,1
2,2DDSM,Data_image/0029DDSM/CC/R/CC_R_2DDSM.png,ddsm,1
3,3DDSM,Data_image/0029DDSM/MLO/R/MLO_R_3DDSM.png,ddsm,1
4,4DDSM,Data_image/0033DDSM/CC/L/CC_L_4DDSM.png,ddsm,1
...,...,...,...,...
7803,7803DDSM,Data_image/4607DDSM/MLO/R/MLO_R_7803DDSM.png,ddsm,0
7804,7804DDSM,Data_image/4608DDSM/CC/L/CC_L_7804DDSM.png,ddsm,0
7805,7805DDSM,Data_image/4608DDSM/MLO/L/MLO_L_7805DDSM.png,ddsm,0
7806,7806DDSM,Data_image/4608DDSM/CC/R/CC_R_7806DDSM.png,ddsm,0


In [63]:
cdd

,image_id,Path,Type,status
0,P1-L-DM-MLO,Data_image/1CDD/MLO/L/MLO_L_P1-L-DM-MLO.png,cdd,2
1,P2-R-DM-CC,Data_image/2CDD/CC/R/CC_R_P2-R-DM-CC.png,cdd,1
2,P2-R-DM-MLO,Data_image/2CDD/MLO/R/MLO_R_P2-R-DM-MLO.png,cdd,1
3,P2-L-DM-CC,Data_image/2CDD/CC/L/CC_L_P2-L-DM-CC.png,cdd,1
4,P2-L-DM-MLO,Data_image/2CDD/MLO/L/MLO_L_P2-L-DM-MLO.png,cdd,1
...,...,...,...,...
1061,P316-L-DM-MLO,Data_image/316CDD/MLO/L/MLO_L_P316-L-DM-MLO.png,cdd,2
1062,P319-R-DM-CC,Data_image/319CDD/CC/R/CC_R_P319-R-DM-CC.png,cdd,1
1063,P319-R-DM-MLO,Data_image/319CDD/MLO/R/MLO_R_P319-R-DM-MLO.png,cdd,1
1064,P325-R-DM-CC,Data_image/325CDD/CC/R/CC_R_P325-R-DM-CC.png,cdd,1
